In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
from sklearn.preprocessing import OneHotEncoder

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim.lr_scheduler as lr_scheduler

import PIL
from PIL import Image
import PIL.Image as pilimg
import matplotlib.pyplot as plt

import os 
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
# if exist error : use below command
# pip3 install -U scikit-learn

In [3]:
# make directory for save traing weight
PATH = './ckpt_folder/'

if not os.path.isdir(PATH):
    os.mkdir(PATH)

# define the hyper parameters for training
nb_batch = 8
nb_epochs = 50
lr = 1e-4
validation_split = 0.2
random_seed= 42
shuffle_dataset = True # shuffle dataset from training data for splitted dataset : entire_train = train + valid

In [4]:
# define the dataset loader ===============================

In [5]:
# Dataset 상속
class CustomDataset(Dataset): 
    def __init__(self, data_dir='train/', class_num_1=13, class_num_2=20):
        self.dir = data_dir
        self.data_pose = data_dir + 'train.tsv'
        # Need zero class >> plus one part
        self.class_num_1 = class_num_1 + 1 # number of first label
        self.class_num_2 = class_num_2 + 1 # number of second label
        self.enc = OneHotEncoder
        
        dataset = []
        for line in open(self.data_pose,'r'):
            spl = line.strip().split('\t')
            dataset.append([spl[0], spl[1], spl[2]])
        
        dataset_np = np.array(dataset)
        
        self.img_name = dataset_np[:,0]
        self.cls_1 = dataset_np[:,1].astype(np.int)
        self.cls_2 = dataset_np[:,2].astype(np.int)
        
    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.img_name)

    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        # image part
        img_name = self.dir + self.img_name[idx]
        img_data = pilimg.open(img_name)
        img_arr = np.array(img_data) / 255.
        # transform img_arr from [height, width, depth] to [depth, height, width]
        x_input = torch.FloatTensor( np.transpose(img_arr, (2, 0, 1)) )
        
        # label - 1 part
        y1_lab_onehot = np.zeros(shape=(self.class_num_1,), dtype=np.int8)
        y1_lab_onehot[self.cls_1[idx]] = 1
        
        # label - 2 part
        y2_lab_onehot = np.zeros(shape=(self.class_num_2,), dtype=np.int8)
        y2_lab_onehot[self.cls_2[idx]] = 1
        
        x = torch.FloatTensor(x_input)
        y1 = torch.FloatTensor(y1_lab_onehot)
        y2 = torch.FloatTensor(y2_lab_onehot)
        return x, y1, y2

In [6]:
# split and shuffling reference site : 
# https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

In [7]:
# training data loader
dataset_train = CustomDataset()
dataset_size = len(dataset_train)

indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

#train_loader, valid_loader = torch.utils.data.random_split(dataset_train.dataset, batch_size=nb_batch, sampler=[train_sampler, valid_sampler])

train_loader = DataLoader(dataset_train, batch_size=nb_batch, sampler=train_sampler)
valid_loader = DataLoader(dataset_train, batch_size=nb_batch, sampler=valid_sampler)

In [8]:
# define the models for training ===============================

In [9]:
class _res_block(nn.Module):
    def __init__(self, in_channels):
        super(_res_block, self).__init__()
        
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True)
        )
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        out = self.block_1(x)
        out = self.block_2(out)
        return out

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn_1 = nn.BatchNorm2d(64)
        self.relu_1 = nn.ReLU()
        
        self.conv_2 = _res_block(in_channels=64)        
        self.maxpool = nn.MaxPool2d(2)
        
        self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn_3 = nn.BatchNorm2d(128)
        self.relu_3 = nn.ReLU()
        
        self.conv_4 = _res_block(in_channels=128)
        
        self.conv_5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn_5 = nn.BatchNorm2d(256)
        self.relu_5 = nn.ReLU()
        
        self.conv_6 = _res_block(in_channels=256)
        
        self.conv_7 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn_7 = nn.BatchNorm2d(512)
        self.relu_7 = nn.ReLU()
        
        self.conv_8 = _res_block(in_channels=512)
        
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        self.fc_1 = nn.Linear(512, 14)
        self.fc_2 = nn.Linear(512, 21)
        
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        
        # stage - 1
        out = self.bn_1(self.conv_1(x))
        out = self.relu_1(out)
        out = self.conv_2(out)
        out = self.maxpool(out)
        
        # stage - 2
        out = self.bn_3(self.conv_3(out))
        out = self.relu_3(out)
        out = self.conv_4(out)
        out = self.maxpool(out)
        
        # stage - 3
        out = self.bn_5(self.conv_5(out))
        out = self.relu_5(out)
        out = self.conv_6(out)
        out = self.maxpool(out)
        
        # stage - 4
        out = self.bn_7(self.conv_7(out))
        out = self.relu_7(out)
        out = self.conv_8(out)
        
        # glocal average pooling 
        out = self.gap(out)
        
        # fully connected layer and soft-max
        out_flat = out.view(out.size(0), -1)
        out_1 = self.fc_1(out_flat)
        out_2 = self.fc_2(out_flat)
        
        return self.softmax(out_1), self.softmax(out_2)
        #return out_1, out_2

In [11]:
def CE_one_hot_to_num(input_onehot):
    _, input_num = input_onehot.max(dim=1)
    return input_num

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Net().to(device)
#criterion = F.mse_loss
#criterion = F.cross_entropy # >> Not work
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5,10,15,20,25,30,35,40,45], gamma= 0.5)

In [13]:
# cross entropy one-hot vector form code reference : 
# https://discuss.pytorch.org/t/cross-entropy-with-one-hot-targets/13580/4

In [14]:
def cross_entropy(pred, target, size_average=True):
    """ Cross entropy that accepts soft targets
    Args:
         pred: predictions for neural network
         targets: targets, can be soft
         size_average: if false, sum is returned instead of mean

    Examples::

        input = torch.FloatTensor([[1.1, 2.8, 1.3], [1.1, 2.1, 4.8]])
        input = torch.autograd.Variable(out, requires_grad=True)

        target = torch.FloatTensor([[0.05, 0.9, 0.05], [0.05, 0.05, 0.9]])
        target = torch.autograd.Variable(y1)
        loss = cross_entropy(input, target)
        loss.backward()
    """
    logsoftmax = nn.LogSoftmax()
    if size_average:
        return torch.mean(torch.sum(-target * logsoftmax(pred), dim=1))
    else:
        return torch.sum(torch.sum(-target * logsoftmax(pred), dim=1))
    
criterion = cross_entropy

In [15]:
# print("start training~!! =========================")

# train_loss_check = []
# loss_set = []
# for epoch in range(nb_epochs):
#     scheduler.step() # apply the learning rate scheduler
    
#     model.train()
#     for batch_idx, samples in enumerate(train_loader):
#         x_train, y_train1, y_train2 = samples
#         data, tar_1, tar_2 = x_train.to(device), y_train1.to(device), y_train2.to(device)
#         optimizer.zero_grad()
#         output_1, output_2 = model(data)
        
#         # loss 
#         loss1 = criterion(target=tar_1, pred=output_1)
#         loss2 = criterion(target=tar_2, pred=output_2)
        
#         loss_val = loss1 + loss2
#         loss_val.backward()
#         optimizer.step()
        
#         train_loss_check.append(loss_val.item())
#         if( batch_idx % 100 == 0 ):
#             print("\t iteration num : {} // train loss value : {:.6f}".format(batch_idx, loss_val.item()))
        
#     print("Epoch num : {} // train loss value : {:.6f}".format(epoch+1, loss_val.item()))
#     PATH_ckpt = PATH + "epoch_" + str(epoch).zfill(3) + '_ckpt.pth'
#     torch.save(model.state_dict(), PATH_ckpt)
    
#     model.eval()
#     with torch.no_grad(): # very very very very important!!!
#         test_loss = []
#         correct_1 = 0
#         correct_2 = 0
#         for batch_idx, samples in enumerate(valid_loader):
#             x_test, y_test1, y_test2 = samples
#             data, tar_1, tar_2 = x_test.to(device), y_test1.to(device), y_test2.to(device)
#             output_1, output_2 = model(data)
            
#             pred_1 = output_1.argmax(dim=1, keepdim=True)
#             pred_2 = output_2.argmax(dim=1, keepdim=True)
            
#             target_1 = tar_1.argmax(dim=1, keepdim=True)
#             target_2 = tar_2.argmax(dim=1, keepdim=True)
            
#             correct_1 += pred_1.eq( target_1.view_as(pred_1)  ).sum().item()
#             correct_2 += pred_2.eq( target_2.view_as(pred_2)  ).sum().item()
            
#             # loss value check
#             loss1 = criterion(target=tar_1, pred=output_1)
#             loss2 = criterion(target=tar_2, pred=output_2)

#             loss_val = loss1 + loss2
#             test_loss.append(loss_val.item())
            
#         corr_1_val = (100. * correct_1) / len(valid_sampler)
#         corr_2_val = (100. * correct_2) / len(valid_sampler)

#         print("validation loss : {:.6f}".format( np.mean(test_loss) ))
#         print("validation accuracy : case - 1 : {:.6f} // case-2 : {:.6f}".format(corr_1_val, corr_2_val) )
        
#     loss_set.append([ np.mean(train_loss_check), np.mean(test_loss)])

start training~!! =========================


/data1/home/hoyin/anaconda3-1/envs/bear_solaris/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


	 iteration num : 0 // train loss value : 5.680109
	 iteration num : 100 // train loss value : 5.390731
	 iteration num : 200 // train loss value : 5.055564
	 iteration num : 300 // train loss value : 4.721043
	 iteration num : 400 // train loss value : 5.320446
	 iteration num : 500 // train loss value : 4.916174
	 iteration num : 600 // train loss value : 5.003072
	 iteration num : 700 // train loss value : 5.058442
	 iteration num : 800 // train loss value : 4.768394
	 iteration num : 900 // train loss value : 5.383141
	 iteration num : 1000 // train loss value : 5.067809
	 iteration num : 1100 // train loss value : 4.886039
	 iteration num : 1200 // train loss value : 5.104815
	 iteration num : 1300 // train loss value : 4.999990
	 iteration num : 1400 // train loss value : 4.749073
	 iteration num : 1500 // train loss value : 5.010344
Epoch num : 1 // train loss value : 5.120950
validation loss : 4.966242
validation accuracy : case - 1 : 58.812500 // case-2 : 36.406250
	 iteration

KeyboardInterrupt: 

In [ ]:
# prev result : validation accuracy : case - 1 : 85.968750 // case-2 : 66.281250

In [ ]:
loss_set_arr = np.array(loss_set)

fig = plt.figure(figsize=(10,10))
plt.subplot(1,1,1)
plt.plot(loss_set_arr[:,0])
plt.title('train loss ', fontsize=10)

plt.subplot(1,2,1)
plt.plot(loss_set_arr[:,1])
plt.title('test loss ', fontsize=10)

plt.show()

In [ ]:
# evaluation part script

In [15]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [16]:
model_path = "dummy/ckpt_folder_0.89_0.74/epoch_009_ckpt.pth"
model.load_state_dict(torch.load(model_path))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [17]:
print("start test~!! =========================")

model.eval()
f1_score_cls1 = 0
f1_score_cls2 = 0

with torch.no_grad(): # very very very very important!!!
    for batch_idx, samples in enumerate(valid_loader):
        x_test, y_test1, y_test2 = samples
        data, tar_1, tar_2 = x_test.to(device), y_test1.to(device), y_test2.to(device)
        output_1, output_2 = model(data)

        pred_1 = output_1.argmax(dim=1, keepdim=True)
        pred_2 = output_2.argmax(dim=1, keepdim=True)

        target_1 = tar_1.argmax(dim=1, keepdim=True)
        target_2 = tar_2.argmax(dim=1, keepdim=True)

        #correct_1 += pred_1.eq( target_1.view_as(pred_1)  ).sum().item()
        #correct_2 += pred_2.eq( target_2.view_as(pred_2)  ).sum().item()
        
        for b_idx in range( len(x_test) ):
            tar_1_onehot = tar_1.cpu().numpy()[b_idx]
            tar_2_onehot = tar_2.cpu().numpy()[b_idx]
            
            # make prediction result as one-hot vector form
            pred_1_idx = pred_1[b_idx]
            pred_2_idx = pred_2[b_idx]
            
            pred_1_onehot = np.zeros(shape=(14,), dtype=np.int8)
            pred_2_onehot = np.zeros(shape=(21,), dtype=np.int8)
            
            pred_1_onehot[pred_1_idx] = 1
            pred_2_onehot[pred_2_idx] = 1
            
            f1_score_cls1 += f1_score( tar_1_onehot, pred_1_onehot )
            f1_score_cls2 += f1_score( tar_2_onehot, pred_2_onehot )
        
    print( f1_score_cls1 / len(valid_loader) ) 
    print( f1_score_cls2 / len(valid_loader) ) 

start test~!! =========================
7.175
5.9225


In [18]:
len(valid_loader)

400

In [18]:
f1_score_cls1

2870.0